<a href="https://colab.research.google.com/github/Namesakenberg/machine_learning/blob/main/KNN_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter


In [2]:
df = pd.read_csv('/content/Social_Network_Ads.csv')

In [3]:
df.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [4]:
df.isnull().sum()

,0
User ID,0
Gender,0
Age,0
EstimatedSalary,0
Purchased,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   User ID          400 non-null    int64 
 1   Gender           400 non-null    object
 2   Age              400 non-null    int64 
 3   EstimatedSalary  400 non-null    int64 
 4   Purchased        400 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 15.8+ KB


In [6]:
df.drop(columns = 'User ID',inplace =True)

In [7]:
df.head()

,Gender,Age,EstimatedSalary,Purchased
0,Male,19,19000,0
1,Male,35,20000,0
2,Female,26,43000,0
3,Female,27,57000,0
4,Male,19,76000,0


In [8]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
df['Gender'] = enc.fit_transform(df['Gender'])

In [9]:
df

,Gender,Age,EstimatedSalary,Purchased
0,1,19,19000,0
1,1,35,20000,0
2,0,26,43000,0
3,0,27,57000,0
4,1,19,76000,0
...,...,...,...,...
395,0,46,41000,1
396,1,51,23000,1
397,0,50,20000,1
398,1,36,33000,0


In [10]:
X = df.drop(columns='Purchased')
y = df['Purchased']

In [11]:
from sklearn.model_selection import train_test_split
X_train , X_test ,y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=0)


In [12]:
X_train

,Gender,Age,EstimatedSalary
336,1,58,144000
64,0,59,83000
55,0,24,55000
106,0,26,35000
300,0,58,38000
...,...,...,...
323,0,48,30000
192,1,29,43000
117,1,36,52000
47,0,27,54000


In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

In [14]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
scores = []
for i in range(1,16):
  knn=KNeighborsClassifier(n_neighbors=i)
  knn.fit(X_train_scaled , y_train)
  y_pred = knn.predict(X_test_scaled)
  acc = accuracy_score(y_test , y_pred)
  scores.append(acc)
best_k = scores.index(max(scores)) + 1
print("Best value of k is:", best_k)
print("Maximum accuracy is:",max(scores))

Best value of k is: 3
Maximum accuracy is: 0.95


In [15]:
class myKnn :
  def __init__(self,k) -> None:
    self.n_neighbors = k
    self.X_train = None
    self.y_train = None


  def fit(self , X_train , y_train):
    self.X_train = X_train                      # KNN is a lazy learner nothing happens while fitting the model
    self.y_train = y_train


  def predict(self , X_test):
    y_pred = []
    for i in X_test :
      distances = []  # empty list to store the distance of every testing point from every training point
      for j in self.X_train:
        distances.append((self.calculateDistance(i,j)))
      n_neighbors = sorted(list(enumerate(distances)),key=lambda x:x[1])[0:self.n_neighbors] # enumerate is used to store the id and the distance of the point from its
      label = self.majority_count(n_neighbors)
      y_pred.append(label)
    return np.array(y_pred)

  def calculateDistance(self,point_A,point_B):
    return np.linalg.norm(point_A - point_B)            # calculates the euclidian distance between 2 poinits using norm function in numpy

  def majority_count(self,neighbors):
    votes = []
    for i in neighbors:
      votes.append(self.y_train[i[0]])    # to get the class of the datapoint put the tuple of datapoint and its distance to y_train
                                          # since i[0] is used the index of the datapoint is only passed to y_train and not the y_train
      vote_count = Counter(votes)              # Common returns a tuple having the most occuring class
      return vote_count.most_common()[0][0]
